In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [3]:
reviews_url = 'https://www.amazon.in/i7-14700K-Desktop-Processor-Integrated-Graphics/product-reviews/B0CGJ41C9W/ref=cm_cr_getr_d_paging_btm_next_5?ie=UTF8&reviewerType=all_reviews&filterByStar=five_star&pageNumber=5'

In [4]:
len_page = 4

In [5]:
def reviewsHtml(url, len_page):
    
    # Empty List define to store all pages html data
    soups = []
    
    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):
        
        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        # Request make for each page
        response = requests.get(url, headers=headers, params=params)
        
        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Add single Html page data in master soups list
        soups.append(soup)
        
    return soups

In [6]:
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []
    
    
    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')
    
    # Iterate all Reviews BOX 
    for box in boxes:
        
        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data 
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)
    
    return data_dicts

In [7]:
html_datas = reviewsHtml(reviews_url, len_page)

In [8]:
html_datas = reviewsHtml(reviews_url, len_page)

In [9]:
reviews = []

In [10]:
for html_data in html_datas:
    
    # Grab review data
    review = getReviews(html_data)
    
    # add review data in reviews empty list
    reviews += review

In [11]:
df_reviews = pd.DataFrame(reviews)

In [12]:
df_reviews

,Name,Stars,Title,Date,Description
0,Tatiana Lopez,N/A,I MADE IT WORK,N/A,This is something that a lot of people do not ...
1,Yuri,N/A,Poderosa para gaming,N/A,Bueno vengo cambiando de una 12400F a 14600KF ...
2,Oneil Alberto Cuevas,N/A,Blazing Fast Performance with Intel Core i9-14...,N/A,The Intel Core i9-14900K processor is a beast ...
3,King Jaja,N/A,"A beast! Awesome, very powerful indeed.",N/A,This thing is a beast! Very powerful indeed. ...
4,Amazon Customer,N/A,Fast!,N/A,It is very fast but it also runs very hot when...
5,Amazon Customer,N/A,Good CPU.,N/A,Be careful with cooling. Try to keep under 75 ...
6,Esper,N/A,Killer Processor,N/A,In 2020 I made the mistake of switching to AMD...
7,Nonsha,N/A,Thumbs Up,N/A,The only reason I knocked down the rating unde...
8,Sims,N/A,Top of the line,N/A,"Great processor, well packaged. I like the waf..."
9,Cole P.,N/A,Super Powerful CPU,N/A,Works great in new build! Only slight issue is...


In [15]:
df_reviews.to_csv('reviews91_star5.csv', index=False)